# Neural Network

In [1]:
# STANDARD
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# SKYLEARN
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
import numpy as np

# TENSORFLOW
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# DUMMY ENCODING
import warnings
warnings.simplefilter('ignore')

In [2]:
# Read CSV, zero all NaN
data = pd.read_csv('data/Combined_data_annual only.csv')
data = data.fillna(0)
data

,Date,CS_Ann,CD_Ann,Enrgy_Ann,Fncl_Ann,HlthC_Ann,Indstrl_Ann,Mtrls_Ann,Tech_Ann,Util_Ann,Unempl,Jobs_Non-Farm (000s),Hrly_Wages,CPI_urban,PPI_Commodity,Imp_Exp _Indx,haircut_price,cardboard_pice
0,12/1/98,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.044,127596,0.00,164.400,0.0,91.0,102.70,166.5
1,1/1/99,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.043,127702,0.00,164.700,0.0,91.3,103.00,165.0
2,2/1/99,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.044,128120,0.00,164.700,0.0,91.2,103.30,164.5
3,3/1/99,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.042,128227,0.00,164.800,0.0,91.5,103.70,164.6
4,4/1/99,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.043,128597,0.00,165.900,0.0,92.4,103.90,171.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
258,6/1/20,0.0641,0.1235,-0.3460,-0.0951,0.1717,-0.0602,0.0486,0.3543,0.0210,0.111,137809,29.32,257.214,117.5,120.6,172.08,358.9
259,7/1/20,0.0065,0.0595,-0.4109,-0.1676,0.0768,-0.1111,-0.0428,0.3098,-0.0535,0.102,139570,29.35,258.723,118.0,122.1,174.74,358.9
260,8/1/20,0.0525,0.1363,-0.4191,-0.1452,0.1650,-0.0754,0.0412,0.3834,0.0225,0.084,141063,29.45,259.681,118.4,123.1,174.26,358.9
261,9/1/20,0.0845,0.2623,-0.3724,-0.0663,0.2004,0.0336,0.1198,0.5806,-0.0559,0.079,141735,29.46,260.209,118.9,123.3,174.75,358.9


In [3]:
# Add new column, "Buy"/"Sell" function
funcs = []

for x in data["CS_Ann"]:
    if x <= 0:
        func = 0
        funcs.append(func)
    elif x > 0:
        func = 1
        funcs.append(func)
    
data["CS_BS"] = funcs
data

,Date,CS_Ann,CD_Ann,Enrgy_Ann,Fncl_Ann,HlthC_Ann,Indstrl_Ann,Mtrls_Ann,Tech_Ann,Util_Ann,Unempl,Jobs_Non-Farm (000s),Hrly_Wages,CPI_urban,PPI_Commodity,Imp_Exp _Indx,haircut_price,cardboard_pice,CS_BS
0,12/1/98,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.044,127596,0.00,164.400,0.0,91.0,102.70,166.5,0
1,1/1/99,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.043,127702,0.00,164.700,0.0,91.3,103.00,165.0,0
2,2/1/99,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.044,128120,0.00,164.700,0.0,91.2,103.30,164.5,0
3,3/1/99,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.042,128227,0.00,164.800,0.0,91.5,103.70,164.6,0
4,4/1/99,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.043,128597,0.00,165.900,0.0,92.4,103.90,171.3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
258,6/1/20,0.0641,0.1235,-0.3460,-0.0951,0.1717,-0.0602,0.0486,0.3543,0.0210,0.111,137809,29.32,257.214,117.5,120.6,172.08,358.9,1
259,7/1/20,0.0065,0.0595,-0.4109,-0.1676,0.0768,-0.1111,-0.0428,0.3098,-0.0535,0.102,139570,29.35,258.723,118.0,122.1,174.74,358.9,1
260,8/1/20,0.0525,0.1363,-0.4191,-0.1452,0.1650,-0.0754,0.0412,0.3834,0.0225,0.084,141063,29.45,259.681,118.4,123.1,174.26,358.9,1
261,9/1/20,0.0845,0.2623,-0.3724,-0.0663,0.2004,0.0336,0.1198,0.5806,-0.0559,0.079,141735,29.46,260.209,118.9,123.3,174.75,358.9,1


In [4]:
data[data["CS_Ann"] < 0]

,Date,CS_Ann,CD_Ann,Enrgy_Ann,Fncl_Ann,HlthC_Ann,Indstrl_Ann,Mtrls_Ann,Tech_Ann,Util_Ann,Unempl,Jobs_Non-Farm (000s),Hrly_Wages,CPI_urban,PPI_Commodity,Imp_Exp _Indx,haircut_price,cardboard_pice,CS_BS
12,12/1/99,-0.0441,0.1358,0.1555,0.0313,0.1474,0.1879,0.1564,0.4349,-0.0444,0.040,130772,0.00,168.800,0.0,97.4,106.30,191.9,0
13,1/1/00,-0.1553,0.1852,0.1732,0.0243,0.1810,0.1989,0.2158,0.7033,-0.0359,0.040,131005,0.00,169.300,0.0,97.8,106.70,191.9,0
14,2/1/00,-0.1459,-0.0057,0.2521,-0.0313,0.1039,0.1084,0.0962,0.3239,-0.0695,0.041,131124,0.00,170.000,0.0,99.7,107.20,191.5,0
15,3/1/00,-0.2271,-0.0557,0.2004,-0.1451,0.0385,0.0504,-0.0022,0.6654,-0.1154,0.040,131596,0.00,171.000,0.0,99.9,107.50,192.3,0
16,4/1/00,-0.1906,0.0315,0.1980,-0.0038,0.1044,0.1740,0.0760,0.5950,-0.0058,0.038,131888,0.00,170.900,0.0,98.5,107.50,197.5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236,8/1/18,-0.0356,0.2155,0.1469,0.1136,0.1085,0.1208,0.0862,0.2433,-0.0152,0.038,149268,27.22,251.905,116.5,127.6,161.73,362.5,0
237,9/1/18,-0.0202,0.2997,0.1857,0.1431,0.1387,0.1195,0.0697,0.2759,-0.0276,0.037,149348,27.31,252.261,116.7,127.7,161.87,363.4,0
241,1/1/19,-0.1178,-0.0181,-0.2228,-0.1645,0.0264,-0.1641,-0.1819,-0.0545,0.0008,0.040,150134,27.58,252.550,117.0,124.6,163.86,361.5,0
242,2/1/19,-0.0703,0.0066,-0.1398,-0.1247,0.0344,-0.0914,-0.1470,-0.0277,0.0696,0.038,150135,27.69,253.181,117.2,125.9,164.32,361.5,0


In [5]:
kevin_input=data.drop(['CS_BS', 'Date'], axis=1)

In [6]:
input_cols=['Unempl',
       'Jobs_Non-Farm (000s)', 'Hrly_Wages', 'CPI_urban', 'PPI_Commodity',
       'Imp_Exp _Indx', 'haircut_price', 'cardboard_pice']
input_df=kevin_input[input_cols]
target_df=kevin_input['CS_Ann']
predicted=[]
from sklearn.tree import DecisionTreeRegressor
initial_train=30
num_runs=len(input_df)-initial_train
for each_cycle in range(num_runs):
    dtr=DecisionTreeRegressor()
    dtr.fit(input_df.iloc[:initial_train+each_cycle, :], target_df[:initial_train+each_cycle])
    prediction=rf.predict([input_df.iloc[initial_train+each_cycle, :]])
    print(f'Predict: {prediction} when actual is {target_df[initial_train+each_cycle]}')

NameError: name 'rf' is not defined

In [ ]:
data.columns

In [ ]:
data.dtypes

### Start here with each new sector column

In [ ]:
# train, test, split
X = data[['CD_Ann', 'Enrgy_Ann', 'Fncl_Ann', 'HlthC_Ann',
       'Indstrl_Ann', 'Mtrls_Ann', 'Tech_Ann', 'Util_Ann', 'Unempl',
       'Jobs_Non-Farm (000s)', 'Hrly_Wages', 'CPI_urban', 'PPI_Commodity',
       'Imp_Exp _Indx', 'haircut_price', 'cardboard_pice']]
y = data["CS_BS"].values.reshape(-1,1)
print(X.shape, y.shape)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [ ]:
y_train

In [ ]:
X_test.shape[0] + X_train.shape[0]

In [ ]:
X_test.head()

## Data Preprocessing

Scale data prior to multilayer perceptron models.

In [ ]:
y_train

In [ ]:
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
y_train_scaled = y_scaler.transform(y_train)
y_test_scaled = y_scaler.transform(y_test)

In [ ]:
X_train.head(4)

In [ ]:
X_train_scaled[0]

In [ ]:
X_train_scaled[1]

Scale both the training and testing data

## Model

Sequential model

In [ ]:
def count_trues(list1, list2):

    count_of_trues = []

    for i in range(len(list1)):
        if list1[i] == list2[i]:
            count_of_trues.append(1)
    return len(count_of_trues)

In [ ]:
# optimization would be to look at the penalty parameter and C (with the lambda)

model = LogisticRegression()
model.fit(X_train_scaled, y_train_scaled)
y_pred_train = model.predict(X_train_scaled)
y_pred_test = model.predict(X_test_scaled)

In [ ]:
print("Percent Accuracy of Training Set:", count_trues(y_train_scaled, y_pred_train) / len(y_train_scaled)) 
print("Percent Accuracy of Test Set:", count_trues(y_test_scaled, y_pred_test) / len(y_test_scaled))



# CD Sector

In [ ]:
# train, test, split
X = data[['CD_Ann', 'Enrgy_Ann', 'Fncl_Ann', 'HlthC_Ann',
       'Indstrl_Ann', 'Mtrls_Ann', 'Tech_Ann', 'Util_Ann', 'Unempl',
       'Jobs_Non-Farm (000s)', 'Hrly_Wages', 'CPI_urban', 'PPI_Commodity',
       'Imp_Exp _Indx', 'haircut_price', 'cardboard_pice']]
y = data["CS_BS"].values.reshape(-1,1)
print(X.shape, y.shape)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [ ]:
y_train

In [ ]:
X_test.shape[0] + X_train.shape[0]

In [ ]:
X_test.head()

## Data Preprocessing

Scale data prior to multilayer perceptron models.

In [ ]:
y_train

In [ ]:
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
y_train_scaled = y_scaler.transform(y_train)
y_test_scaled = y_scaler.transform(y_test)

In [ ]:
X_train.head(4)

In [ ]:
X_train_scaled[0]

In [ ]:
X_train_scaled[1]

Scale both the training and testing data

## Model

Sequential model

In [ ]:
def count_trues(list1, list2):

    count_of_trues = []

    for i in range(len(list1)):
        if list1[i] == list2[i]:
            count_of_trues.append(1)
    return len(count_of_trues)

In [ ]:
# optimization would be to look at the penalty parameter and C (with the lambda)

model = LogisticRegression()
model.fit(X_train_scaled, y_train_scaled)
y_pred_train = model.predict(X_train_scaled)
y_pred_test = model.predict(X_test_scaled)

In [ ]:
print("Percent Accuracy of Training Set:", count_trues(y_train_scaled, y_pred_train) / len(y_train_scaled)) 
print("Percent Accuracy of Test Set:", count_trues(y_test_scaled, y_pred_test) / len(y_test_scaled))



## Quantify the Model
We use our testing data to validate our model. This is how we determine the validity of our model (i.e. the ability to predict new and previously unseen data points)

In [ ]:
# Evaluate the model using testing data
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

## Making Predictions with new data

Make predictions using `model.predict`

In [ ]:
import numpy as np
new_data = np.array([[0.2, 0.3, 0.4]])
print(f"Predicted class: {model.predict_classes(new_data)}")

In [ ]:
# X_train_scaled = X_scaler.transform(X_train)
# X_test_scaled = X_scaler.transform(X_test)

In [ ]:
# Encode, scale, and normalize

data = X.copy()

data_binary_encoded = pd.get_dummies(data)
data_binary_encoded.head()

X = pd.get_dummies(X)

X_train, X_test, y_train, y_test = train_test_split(X, y)

X_train.head()


# STANDARD SCALER
X_scaler = StandardScaler().fit(X_train)
y_scaler = StandardScaler().fit(y_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
y_train_scaled = y_scaler.transform(y_train)
y_test_scaled = y_scaler.transform(y_test)

One-hot encode the labels

# One-hot encoding
y_train_categorical = to_categorical(y_train)
y_test_categorical = to_categorical(y_test)
y_train_categorical